In [ ]:
%pip install setfit[optuna]==0.3.0 datasets -U

  Using cached datasets-2.20.0-py3-none-any.whl (547 kB)


In [ ]:
import pandas as pd

In [ ]:
data_path = pd.read_csv('/content/Clean_train.csv')

In [ ]:
#data_path.dropna(inplace = True)

In [ ]:
seed = 20
labels = 2
samples_per_labels = 10
sampled_datasets = []

In [ ]:
from datasets import load_dataset , concatenate_datasets

In [ ]:
# Initialize a list to store sampled datasets
sampled_datasets = []

# Iterate over labels 0 and 1
for i in range(2):

    filtered_data = data_path[data_path["target"] == i]


    if not filtered_data.empty:
        sampled_data = filtered_data.sample(n=samples_per_labels, random_state=seed, replace=True)
        sampled_datasets.append(sampled_data)
    else:
        print(f"No data found for label {i}")


In [ ]:
train_dataset = pd.concat(sampled_datasets, ignore_index=True)

In [ ]:
test_samples_per_label = 1000

# Initialize a list to store sampled test datasets
sampled_test_datasets = []

# Iterate over each label
for i in range(labels):

    filtered_data = data_path[data_path["target"] == i+1]


    if not filtered_data.empty:
        sampled_test_data = filtered_data.sample(n=test_samples_per_label, random_state=seed+1, replace=True)
        sampled_test_datasets.append(sampled_test_data)
    else:
        print(f"No data found for label {i+1}")


test_dataset = pd.concat(sampled_test_datasets, ignore_index=True)

No data found for label 2


In [ ]:
train_dataset = train_dataset.sample(frac=1, random_state=seed).reset_index(drop=True)

In [ ]:
train_dataset

,Unnamed: 0,id,keyword,location,text,target
0,3955,5622,flooding,NaN,flooding kill displace one million Pakistan Da...,1
1,5740,8191,rescuers,"Here, there and everywhere",video pick body water rescuer search hundred m...,1
2,6019,8598,seismic,NaN,SEISMIC risk comparison case study CALABRIA MA...,1
3,5559,7934,rainstorm,NaN,CalumSOS look like get catch rainstorm amazing...,0
4,2300,3300,demolish,sweden,add video YouTube playlist http tcoKBzUaTUkS D...,0
5,3526,5040,eyewitness,"Washington, DC",anniversary Hiroshima bombing illustrate timel...,1
6,7537,10776,wreckage,India,wreckage Conclusively Confirmed MH Malaysia pm,1
7,7041,10088,typhoon,NaN,Obama declare Disaster Typhoon Devastated Saip...,1
8,7460,10676,wounds,?Gangsta OC / MV RP; 18+.?,icymagistrate her upper arm friggin icicle pro...,1
9,5487,7830,quarantine,"NYC, New York",reddit Quarantine Offensive Content reddit co ...,0


In [ ]:
test_dataset = test_dataset.sample(frac=1, random_state=seed).reset_index(drop=True)

print(test_dataset.head())


   Unnamed: 0    id     keyword                  location  \
0        3354  4800   evacuated  Hensley Street, Portland   
1        5579  7960       razed                       NaN   
2        3521  5034  eyewitness                       NaN   
3        2723  3911  devastated                Birmingham   
4        1151  1658     bombing                       NaN   

                                                text  target  
0  katunew sr remain closed brush fire burn acre ...       1  
1  late home raze Northern California Wildfire AB...       1  
2  little boy affect People Hiroshima Eyewitness ...       1  
3  richmond Coaches devastate hear death second d...       1  
4  Japan mark th anniversary Hiroshima atomic bom...       1  


In [ ]:
from datasets import Dataset

# Convert pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_dataset)


In [ ]:
from datasets import Dataset

# Convert pandas DataFrame to Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_dataset)


In [ ]:
train_dataset = train_dataset.rename_column('target', 'label')

In [ ]:
test_dataset = test_dataset.rename_column('target','label')

In [ ]:
train_dataset

Dataset({
    features: ['Unnamed: 0', 'id', 'keyword', 'location', 'text', 'label'],
    num_rows: 20
})

In [ ]:
test_dataset

Dataset({
    features: ['Unnamed: 0', 'id', 'keyword', 'location', 'text', 'label'],
    num_rows: 1000
})

In [ ]:
!pip install sentence_transformers

In [ ]:
from setfit import SetFitModel , SetFitTrainer
from sentence_transformers.losses  import CosineSimilarityLoss

In [ ]:
model_id = 'sentence-transformers/all-mpnet-base-v2'
model = SetFitModel.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
trainer = SetFitTrainer(
     model= model ,
     train_dataset = train_dataset,
     eval_dataset = test_dataset ,
     loss_class = CosineSimilarityLoss,
     metric = "accuracy",
     batch_size = 16,
     num_iterations = 50,
     num_epochs = 5,
    column_mapping={"text": "text", "label": "label"}
)

In [ ]:
trainer.train()
metrics = trainer.evaluate()

print(f"accuracy :{metrics['accuracy']}")

Applying column mapping to training dataset
***** Running training *****
  Num examples = 2000
  Num epochs = 5
  Total optimization steps = 125
  Total train batch size = 16


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Iteration:   0%|          | 0/125 [00:00<?, ?it/s]

Applying column mapping to evaluation dataset
***** Running evaluation *****


accuracy :0.703
